In [12]:
%cd '/Users/rafalroman/PycharmProjects/python_data_products_workshop/project_submissions'


/Users/rafalroman/PycharmProjects/python_data_products_workshop/project_submissions


In [20]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
"""
created: 2020-03-22
updated:
author: Rafal
description: import data from lines of json to pandas
"""
import os
import pathlib
import json
import pandas as pd

os.chdir('/Users/rafalroman/PycharmProjects/python_data_products_workshop/project_submissions') 

ROOT_PATH = pathlib.Path().absolute()
FILES_PATH = os.path.join(ROOT_PATH, 'airly_sensors/files/datafiles')

airly_file = os.path.join(FILES_PATH, 'klobuck_szkolna_airly.json')
print(airly_file)

class Airly:
    debug = 0  # useful for development
    columns = {'from':str, 'to':str, 'pm1':float, 'pm25':float, 'pm10':float, 'press':float, 'humid':float, 'temp':float}  # dict of measurements

    def __init__(self, airly_file: str):
        self._airly_file = airly_file
        self._measurements_list = []
        if not Airly.debug:
            return None

        print(self._airly_file)

    def _24hours_history(self, json_string: str) -> list:
        one_day_measure = json.loads(json_string)
        history_list_group = one_day_measure['history']
        history_list = []
        for one_measure in history_list_group:
            values = one_measure['values']
            di_name_value = {}
            for pair in values:
                di_name_value[pair['name']] = pair['value']
                
            li = [
#                 one_measure['fromDateTime'].replace(':00Z',':00.000Z') , #.astype(str),
#                 one_measure['tillDateTime'].replace(':00Z',':00.000Z'), #.astype(str),
                
                pd.to_datetime(one_measure['fromDateTime'].replace(':00Z',':00.000Z'), utc=True),
                pd.to_datetime(one_measure['tillDateTime'].replace(':00Z',':00.000Z'), utc=True), 
                di_name_value['PM1'],
                di_name_value['PM10'],
                di_name_value['PM25'],
                di_name_value['PRESSURE'],
                di_name_value['HUMIDITY'],
                di_name_value['TEMPERATURE'],
            ]
            history_list.append(li)
        return history_list

    def _json_rows2array(self):
        max_process = 10000000
        if Airly.debug:
            max_process = Airly.debug
        counter = 0
        error_counter = 0
        self._measurements_list = []
        with open(self._airly_file) as f:
            for line in f:
                counter += 1
                if counter > max_process:
                    break
                try:
                    self._measurements_list.extend(self._24hours_history(line))
                except Exception:
                    error_counter += 1
                    # print(f'error in line: {counter}')
                    # print(line)
        print(f'Erorr counter = {error_counter}')
        return self._measurements_list

    def _create_df(self) -> pd.DataFrame:
        df = pd.DataFrame()
        df = df.append(pd.DataFrame(self._measurements_list, columns=Airly.columns), ignore_index=True)
        return df

    def import2df(self):
        self._measurements_list = self._json_rows2array()
        result = self._create_df()
        return result


klobuck_file = Airly(airly_file)
df = klobuck_file.import2df()
df


/Users/rafalroman/PycharmProjects/python_data_products_workshop/project_submissions/airly_sensors/files/datafiles/klobuck_szkolna_airly.json
Erorr counter = 57


,from,to,pm1,pm25,pm10,press,humid,temp
0,2018-11-15 06:00:00+00:00,2018-11-15 07:00:00+00:00,17.76,47.25,28.26,1031.99,97.58,5.52
1,2018-11-15 07:00:00+00:00,2018-11-15 08:00:00+00:00,17.15,45.84,27.45,1032.25,96.25,6.03
2,2018-11-15 08:00:00+00:00,2018-11-15 09:00:00+00:00,14.66,39.69,23.74,1032.27,93.25,7.02
3,2018-11-15 09:00:00+00:00,2018-11-15 10:00:00+00:00,10.44,25.81,16.91,1032.26,96.25,7.52
4,2018-11-15 10:00:00+00:00,2018-11-15 11:00:00+00:00,9.19,17.71,13.01,1032.11,82.92,8.92
...,...,...,...,...,...,...,...,...
10459,2020-03-22 01:00:00+00:00,2020-03-22 02:00:00+00:00,6.01,12.75,8.92,1028.24,62.08,-1.95
10460,2020-03-22 02:00:00+00:00,2020-03-22 03:00:00+00:00,4.90,10.43,7.33,1028.24,71.59,-2.59
10461,2020-03-22 03:00:00+00:00,2020-03-22 04:00:00+00:00,5.55,11.63,8.22,1028.46,81.48,-2.97
10462,2020-03-22 04:00:00+00:00,2020-03-22 05:00:00+00:00,7.49,15.70,11.19,1028.75,85.25,-2.97


In [86]:
pd.to_datetime('20190102',\
                                format='%Y%m%d')

Timestamp('2019-01-02 00:00:00')

In [70]:
# start = pd.to_datetime('2019-01-02', utc=True)
# end = pd.to_datetime('2019-01-04', utc=True)
# pd.set_option('mode.chained_assignment','raise')
# pd.set_option('mode.chained_assignment','warn')
pd.set_option('mode.chained_assignment', None)
start= '2010-01-01'
end = '2020-12-31'
date_range = df[(df['from'] > start) & (df['from'] < end)]
# date_range['from'].month()
date_range['month'] = date_range['from'].dt.month
date_range['year'] = date_range['from'].dt.year
# date_range.loc[:,'miesiac'] = date_range['from'].dt.month

# df.loc[:, 'colname'] = something
date_range
# dir(date_range['from'].dt)

# date_range.count()
# pd.DatetimeIndex(df['ArrivalDate']).month


,from,to,pm1,pm25,pm10,press,humid,temp,month,year
0,2018-11-15 06:00:00+00:00,2018-11-15 07:00:00+00:00,17.76,47.25,28.26,1031.99,97.58,5.52,11,2018
1,2018-11-15 07:00:00+00:00,2018-11-15 08:00:00+00:00,17.15,45.84,27.45,1032.25,96.25,6.03,11,2018
2,2018-11-15 08:00:00+00:00,2018-11-15 09:00:00+00:00,14.66,39.69,23.74,1032.27,93.25,7.02,11,2018
3,2018-11-15 09:00:00+00:00,2018-11-15 10:00:00+00:00,10.44,25.81,16.91,1032.26,96.25,7.52,11,2018
4,2018-11-15 10:00:00+00:00,2018-11-15 11:00:00+00:00,9.19,17.71,13.01,1032.11,82.92,8.92,11,2018
...,...,...,...,...,...,...,...,...,...,...
10459,2020-03-22 01:00:00+00:00,2020-03-22 02:00:00+00:00,6.01,12.75,8.92,1028.24,62.08,-1.95,3,2020
10460,2020-03-22 02:00:00+00:00,2020-03-22 03:00:00+00:00,4.90,10.43,7.33,1028.24,71.59,-2.59,3,2020
10461,2020-03-22 03:00:00+00:00,2020-03-22 04:00:00+00:00,5.55,11.63,8.22,1028.46,81.48,-2.97,3,2020
10462,2020-03-22 04:00:00+00:00,2020-03-22 05:00:00+00:00,7.49,15.70,11.19,1028.75,85.25,-2.97,3,2020


In [87]:
# date_range.groupby(['year','month'])['temp','humid'].mean()
date_range[['temp','humid','year','month']].groupby(['year','month']).agg(["mean", "median", 'max', 'min'])
# date_range.groupby(['year','month']).agg(['max', 'min'])

temp                            humid                       
                 mean  median    max    min       mean  median     max    min
year month                                                                   
2018 11      0.176693  -0.475   9.08  -6.44  85.154339  87.585   97.75  46.60
     12      0.759543   0.855   8.26  -7.21  90.712003  92.080  100.00  68.13
2019 1      -2.335333  -1.925   4.69 -10.32  88.690389  89.340  100.00  67.54
     2       2.945926   2.740  17.65  -8.51  80.694151  84.250  100.00  32.28
     3       5.454059   5.060  20.81  -7.13  75.721169  76.290  100.00  22.96
     4      10.095000   9.620  27.83  -3.15  63.008431  60.645  100.00  20.33
     5      11.875690  11.235  26.10   1.48  79.522198  86.940  100.00  31.75
     6      21.782271  21.565  32.88   8.30  70.808430  72.810  100.00  29.37
     8      20.451463  19.955  33.23   8.30  72.266883  75.990  100.00  30.03
     9      14.066708  13.505  31.93   2.74  80.083208  86.190  100.00  32.51
     10     10.778333  10.265  25.83  -5.72  81.820972  86.500  100.00  30.02
     11      6.293569   6.470  18.31  -5.72  87.765125  90.315  100.00  45.70
     12      2.662245   2.610  14.22  -3.80  85.124664  86.560  100.00  48.08
2020 1       0.988879   1.100   9.83  -4.50  84.440737  86.500  100.00  42.74
     2       3.392667   3.045  12.77  -4.80  78.957228  81.880  100.00  42.07
     3       5.930824   5.360  19.14  -4.13  72.024529  77.355  100.00  19.79

In [19]:
df.where(df['from'].str.strip().between('2020-01-01', '2020-12-31')).groupby('month')['temp'].mean()

AttributeError: Can only use .str accessor with string values!